# Azure OpenAI Assistants

Azure OpenAI Assistants allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions. In this article we'll provide an in-depth walkthrough of getting started with the Assistants API.

In [1]:
%pip install openai --upgrade
%pip install tiktoken
%pip install numpy
%pip install python-dotenv
%pip install pillow
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv("azure.env")

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model=os.getenv("MODEL_VERSION") #You must replace this value with the deployment name of your model.
)

In [3]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_6HCrt55wdnYo2WwkmTQS4bjN",
  "created_at": 1724859451,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4-2024-04-09",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": null,
  "top_p": 1.0,
  "file_ids": []
}


In [4]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_zgPCQVHvE23CGL9Q2fmd5c6F', created_at=1724859451, metadata={}, object='thread', tool_resources=None)


In [5]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_qzy7vEbQIRh99LM02yooLW3k",
      "assistant_id": null,
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1724859452,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_zgPCQVHvE23CGL9Q2fmd5c6F",
      "file_ids": []
    }
  ],
  "object": "list",
  "first_id": "msg_qzy7vEbQIRh99LM02yooLW3k",
  "last_id": "msg_qzy7vEbQIRh99LM02yooLW3k",
  "has_more": false
}


We could also pass an instructions parameter here, but this would override the existing instructions that we have already provided for the assistant.

In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [13]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [14]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_jY7WXznrSyzTU4K7LHu1AQUc",
      "assistant_id": "asst_6HCrt55wdnYo2WwkmTQS4bjN",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-hSONDnpu0ijAMsZFYs3ou9yI",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's the visualization of the sinewave, represented by the function \\( y = \\sin(x) \\) over the range of \\(0\\) to \\(2\\pi\\). If you need any modifications or additional information on this plot, feel free to ask!"
          },
          "type": "text"
        }
      ],
      "created_at": 1724859474,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_DvGMo7lZ8RL7F8pI5S5aq5t7",
      "status": null,
      "thread_id":

In [15]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(data)  

{'data': [{'id': 'msg_jY7WXznrSyzTU4K7LHu1AQUc', 'assistant_id': 'asst_6HCrt55wdnYo2WwkmTQS4bjN', 'attachments': None, 'completed_at': None, 'content': [{'image_file': {'file_id': 'assistant-hSONDnpu0ijAMsZFYs3ou9yI', 'detail': None}, 'type': 'image_file'}, {'text': {'annotations': [], 'value': "Here's the visualization of the sinewave, represented by the function \\( y = \\sin(x) \\) over the range of \\(0\\) to \\(2\\pi\\). If you need any modifications or additional information on this plot, feel free to ask!"}, 'type': 'text'}], 'created_at': 1724859474, 'incomplete_at': None, 'incomplete_details': None, 'metadata': {}, 'object': 'thread.message', 'role': 'assistant', 'run_id': 'run_DvGMo7lZ8RL7F8pI5S5aq5t7', 'status': None, 'thread_id': 'thread_zgPCQVHvE23CGL9Q2fmd5c6F', 'file_ids': []}, {'id': 'msg_bFH04kurPWp4SCb7J5J7VcSY', 'assistant_id': 'asst_6HCrt55wdnYo2WwkmTQS4bjN', 'attachments': None, 'completed_at': None, 'content': [{'text': {'annotations': [], 'value': "To create a vi

In [16]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [17]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [18]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [19]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)


In [22]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
status = run.status
print(status)

completed


* Please wait until you get "completed"

In [24]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_U6tdtzRE7SIa2xO36OWxXMkJ",
      "assistant_id": "asst_6HCrt55wdnYo2WwkmTQS4bjN",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! Here's the Python code I used to generate and plot the sinewave:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate data points\nx = np.linspace(0, 2*np.pi, 500)\ny = np.sin(x)\n\n# Create the plot\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='y = sin(x)')\nplt.title('Sine Wave')\nplt.xlabel('x')\nplt.ylabel('y')\nplt.grid(True)\nplt.legend()\nplt.show()\n```\n\nThis code utilizes NumPy to create a series of \\( x \\) values and calculates the corresponding \\( y \\) values using the sine function. It then plots these values using matplotlib to visualize the sinewave. If you have any questions or if there's anything else you'd like to see, let me know!"
          },
   

In [25]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Certainly! Here's the Python code I used to generate and plot the sinewave:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data points
x = np.linspace(0, 2*np.pi, 500)
y = np.sin(x)

# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(x, y, label='y = sin(x)')
plt.title('Sine Wave')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()
```

This code utilizes NumPy to create a series of \( x \) values and calculates the corresponding \( y \) values using the sine function. It then plots these values using matplotlib to visualize the sinewave. If you have any questions or if there's anything else you'd like to see, let me know!


In [26]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_6HCrt55wdnYo2WwkmTQS4bjN', deleted=True, object='assistant.deleted')